## COVID19- Argentina


In [1]:
#hide
import os
import numpy as np
import pandas as pd
import glob
import locale
from jinja2 import Template
#
from IPython.display import HTML
# Set Language
_=locale.setlocale(locale.LC_TIME,'es_ES.utf8')

In [2]:
# helpers
def cat_df(data_src):
    lst_=glob.glob(data_src)
    lst_.sort()
    lst_df_=[]
    for l in lst_:
        df_=pd.read_csv(l,parse_dates=[0])
        lst_df_.append(df_)
    df_cat_=pd.concat(lst_df_,axis=0,)
    df_cat_['fecha']=pd.to_datetime(df_cat_['fecha'].dt.strftime('%Y-%m-%d'))
    # Group all with "FIX"
    df_cat_=df_cat_.groupby(['provincia','fecha'])[['confirmados','muertes','recuperados']].sum().reset_index()
    return df_cat_
#
def get_frame_unstack(df_,oncol,group_by_lst=['provincia','fecha']):
    s=df_.groupby(group_by_lst)[oncol].sum().unstack()
    return s

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def write_report(hmtl,out_name):
    with open(out_name,'w') as f:
        f.write(html)
    
#
provDict={'Ciudad Autónoma de Buenos Aire':'CABA'}


In [3]:
# CSV sources
data_src_provincia = '../../data/filled/*provincia*.csv'
data_src_argentina = '../../data/filled/*argentina*.csv'
data_prod = '../../products'
data_includes_ = '../_includes'
os.makedirs(data_prod, exist_ok=True)
os.makedirs(data_includes_, exist_ok=True)
#
df_ = pd.concat([cat_df(data_src_provincia), cat_df(data_src_argentina)])
df_['provincia'] = df_['provincia'].apply(
    lambda x: provDict[x] if x in provDict else x)
df_.head()

,provincia,fecha,confirmados,muertes,recuperados
0,Buenos Aires,2020-03-03,0,0,0
1,Buenos Aires,2020-03-04,0,0,0
2,Buenos Aires,2020-03-05,0,0,0
3,Buenos Aires,2020-03-06,1,0,0
4,Buenos Aires,2020-03-07,0,0,0


In [4]:
# make cumsum on df_
df_cumsum_=df_.groupby(['provincia'])[['confirmados','muertes','recuperados']].cumsum().add_prefix('cumsum_')
# concat dfs
df_cat=pd.concat([df_,df_cumsum_],axis=1)

In [5]:
df_cat

,provincia,fecha,confirmados,muertes,recuperados,cumsum_confirmados,cumsum_muertes,cumsum_recuperados
0,Buenos Aires,2020-03-03,0,0,0,0,0,0
1,Buenos Aires,2020-03-04,0,0,0,0,0,0
2,Buenos Aires,2020-03-05,0,0,0,0,0,0
3,Buenos Aires,2020-03-06,1,0,0,1,0,0
4,Buenos Aires,2020-03-07,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
24,Argentina_Nacion,2020-03-27,101,5,5,690,17,80
25,Argentina_Nacion,2020-03-28,55,2,0,745,19,80
26,Argentina_Nacion,2020-03-29,75,1,0,820,20,80
27,Argentina_Nacion,2020-03-30,146,4,148,966,24,228


In [6]:
import altair as alt
from altair import datum

source=df_cat
circ=alt.Chart(source).mark_circle(color='orange').encode(
    x=alt.X('fecha:T',axis=alt.Axis(title='Fecha'),),
    y=alt.Y('provincia',axis=alt.Axis(title='Provincias/Nación')),
    size='confirmados',
    tooltip=['confirmados', 'fecha','provincia','muertes']
).properties(width=600, height=600).interactive()


In [7]:
circ

alt.Chart(...)

In [8]:
#
pts = alt.selection(type="single", encodings=['x'])
rect=alt.Chart(source).mark_rect().encode(
    alt.Y('provincia:O', title='Provincias/Nacion'),
    alt.X('monthdate(fecha):O', title='Fecha'),
    alt.Color('confirmados:Q', title='Casos Confirmados',),
tooltip=['confirmados', 'fecha','provincia','muertes','recuperados']
).properties(width=600, height=600).interactive()
rect

alt.Chart(...)

In [9]:
pts = alt.selection(type="single", encodings=['x'],empty='none')
#
points = alt.Chart(source).mark_circle(color='orange').encode(
    alt.X('fecha:T',title='Fecha'),
    alt.Y('cumsum_confirmados:Q',scale=alt.Scale(type='log'),title='#Confirmados Totales en Argentina (log scale)'),
    size=alt.Size('confirmados',title='Confirmados x dia'),
    color=alt.condition(pts, 'confirmados:Q', alt.value('orange'),legend=None),
    tooltip=['confirmados', 'fecha','provincia','muertes','recuperados']
).properties(        width=550,
        height=400).transform_filter((datum.provincia == 'Argentina_Nacion')).add_selection(pts)


tooltip_bars=[alt.Tooltip('cumsum_confirmados:Q', title='#Confirmados Totales'),alt.Tooltip('fecha:T', title='Fecha'),alt.Tooltip('provincia:N', title='Provincia')]

bars = alt.Chart(source).mark_bar().encode(
       alt.X('provincia:N',axis=alt.Axis(minExtent=100),title='Provincia de Residencia'),
        alt.Y('cumsum_confirmados:Q',title='Confirmados x Provincia'),
        color=alt.Color('cumsum_confirmados:Q',title='Confirmados x Provincia',legend=None),
        tooltip=tooltip_bars).properties(  
    width=550,
        height=220
    ).transform_filter(datum.provincia != 'Argentina_Nacion').transform_filter((pts)).interactive()

texts = alt.Chart(source).mark_text(dy=0, size=20).encode(
    text='fecha:T'
).transform_filter(
    pts
)


In [10]:
alt.vconcat(points, bars+texts,data=source)

alt.VConcatChart(...)